In [23]:
#Import the necessary libraries
library(tidyverse)
library(dplyr)
library(rpart)
source('functions.R')
library(rpart)

In [24]:
#load the data
train = readRDS("04a-wrangledTrain.rds")
holdout = readRDS('04b-wrangledHoldout.rds')

# 05c Trees
This file will perform k-fold cv on two different tree based models, one on the entire predictor set, the other on a subset of the predictor variables

### Tree based functions
Below you will find the functions FindUniquePos and kFoldTree.
FindUniquePos finds the unique position of an element and is necessary to evaluate the interval score and coverage of tree models.
kFoldTree is a function which performs k fold cross-validation on a tree.

In [25]:
#' @description
#' Find group position for each element of a vector
#'
#' @param values vector which should values that depends on the group
#' @param groupValues group values in the vector (hopefully unique)
#' @param tolerance for matching equality
#'
#' @return group vector with membership label for each element of 'values' #'
FindUniquePos = function(values, groupValues, tolerance=1.e-5){
    ngroup = length(groupValues) # number of groups (terminal nodes)
    temp = unique(groupValues)
    if(length(temp)<ngroup){
        cat("Won't work: non-unique group values\n"); return(0); }
    npred = length(values) # number of cases to bin into a group label group = rep(0,npred) # initialize as group 0
    group = rep(0,npred)
    for(ig in 1:ngroup){
        # group[values==groupValues[i]]=i # better to use tolerance
        igroup = (abs(values-groupValues[ig])<tolerance)
        group[igroup] = ig  # group label according to position in groupValues 
    }
    if( any(group==0) ) cat("Warning: some values not matched to groupValues\n")
    return(group)
    }

The standard kfold function needs to be modified to allow for the interval scores generated by trees.

In [26]:
#' @description
#' Perform Kfold CV on a regression tree
#'
#' @param Kfold integer, the number of folds to perform kfold cv over
#' @param seed integer, a number to set randomness for reproduction
#' @param datafr, a dataframe to train and test the model on, all predictor variables will be used and price will be predicted
#'
#' @return outTree, a report of the performance of the tree on each fold

kFoldTree = function(Kfold, seed, datafr)
{ set.seed(seed)
  n = nrow(datafr)
  iperm<<-sample(n) # set as global for debugging check
  nhold = round(n/Kfold)
  reg = list()
  pred = list() 
  scoreVar = list()
  rocVar = list()
  pred_y = sample(n-nhold)
  results = data.frame(NA,nrow = 3,ncol = 4)
 
  for(k in 1:Kfold){
        ilow = (k-1)*nhold+1
        ihigh = k*nhold
        if(k==Kfold) { ihigh = n }
        ifold = iperm[ilow:ihigh]
        holdo = datafr[ifold,]
        train = datafr[-ifold,]
        RegTree = rpart(log(price)~., data=train)
        meanByTNode = tapply(log(train$price), RegTree$where, mean)
        Q25ByTNode = tapply(log(train$price), RegTree$where, quantile,prob=0.25)
        Q50ByTNode = tapply(log(train$price), RegTree$where, median)
        Q75ByTNode = tapply(log(train$price), RegTree$where, quantile,prob=0.75)
        Q10ByTNode = tapply(log(train$price), RegTree$where, quantile, prob=0.10)
        Q90ByTNode = tapply(log(train$price), RegTree$where, quantile, prob=0.90)
        meanpredRegTree = predict(RegTree, newdata=holdo,type="vector")
        TNodeGroup = FindUniquePos(meanpredRegTree,meanByTNode)
      
        TNodeGroup = FindUniquePos(meanpredRegTree,meanByTNode)
      
        Q25predRegTree = Q25ByTNode[TNodeGroup]; Q75predRegTree = Q75ByTNode[TNodeGroup]
        pred50IntRegTree = exp(cbind(meanpredRegTree,Q25predRegTree,Q75predRegTree))
      
        Q10predRegTree = Q10ByTNode[TNodeGroup]; Q90predRegTree = Q90ByTNode[TNodeGroup] 
        pred80IntRegTree = exp(cbind(meanpredRegTree,Q10predRegTree,Q90predRegTree))
          
        ISTree50 = intervalScore(pred50IntRegTree,holdo$price,0.5)
        ISTree80 = intervalScore(pred80IntRegTree,holdo$price,0.8)
        outTree = rbind(ISTree50$summary,ISTree80$summary)
        colnames(outTree)=c("level","avgleng","IS","cover") 
        print(outTree)

  }
}

### Comparing the Tree
Here I take a subset of the data to isolate the variables which we want to use as predictors to compare with other methods.

In [27]:
kFoldTree(3,123,datafr = train)

     level   avgleng       IS     cover
[1,]   0.5  9477.828 21119.11 0.5004229
[2,]   0.8 20321.567 31628.91 0.7993725
     level   avgleng       IS     cover
[1,]   0.5  9795.193 21610.06 0.4985774
[2,]   0.8 20545.379 32405.93 0.8009874
     level   avgleng       IS     cover
[1,]   0.5  9473.537 21142.21 0.4991541
[2,]   0.8 19925.459 31877.84 0.7981760


This version uses the feature selection function which isolates the predictors age, fuel, drive, type, countryOrigin (manufacturer's home country), isLuxury (luxury brand indicator).

In [29]:
sub_train = feature_selection(train)
kFoldTree(3,123,datafr = sub_train)

     level   avgleng       IS     cover
[1,]   0.5  9620.351 21276.12 0.5020686
[2,]   0.8 20199.253 31864.82 0.7985420
     level   avgleng       IS     cover
[1,]   0.5  9320.868 21214.83 0.4993617
[2,]   0.8 20164.808 31920.39 0.8020486
     level  avgleng       IS     cover
[1,]   0.5  9623.15 21769.10 0.4988773
[2,]   0.8 20584.09 32659.14 0.7968841


### Training the models
Here we train the models and save them for comparison with other methods on the holdout.

In [34]:
tree = rpart(log(price)~., data=train)
subsetTree = rpart(log(price)~., data=feature_selection(train))
saveRDS(tree, "05c-tree.rds")
saveRDS(subsetTree, "05c-subsetTree.rds")